In [8]:
from datasets import load_dataset
from torch.utils.data import DataLoader


In [9]:
import argparse
import datetime
import functools
import json
import os
import random
from typing import Optional, Sequence, Tuple, Type

import numpy as np
import wandb

import torch
import torch.distributed as dist
from torch.distributed.fsdp import (
    FullyShardedDataParallel as FSDP
)
import torch.nn as nn
from torch.utils.data import DataLoader, Subset

from sequence_models.samplers import (
    SortishSampler,
    ApproxBatchSampler,
    ClusteredSortishSampler,
)

from evodiff.utils import Tokenizer
from dayhoff.collators import LMCollator, OAMaskCollator
from dayhoff.constants import MSA_ALPHABET_PLUS, TaskType
from dayhoff.datasets import UniRefDataset
from dayhoff.model import (
    ARDiffusionModel,
    OrderAgnosticDiffusionModel,
)
from dayhoff.model import create_model


# default to a single-GPU setup if not present
RANK = 0
LOCAL_RANK = 0
WORLD_SIZE = 1
DEVICE = torch.device(f"cuda:0")


def is_amlt() -> bool:
    return os.environ.get("AMLT_OUTPUT_DIR", None) is not None


def load_config_and_model(
    config_fpath: str,
) -> Tuple[dict, Tokenizer, nn.Module, Type[nn.Module]]:
    """Parses the experiment config to load the model and tokenizer

    Parameters:
    -----------
    config_fpath: str
        The path to the experiment config file

    Returns:
    --------
    config: dict
        The experiment config
    tokenizer: Tokenizer
        The model's tokenizer
    model: nn.Module
        A task-wrapped version of the specified model, which returns the appropriate loss and metrics
    block: Type[nn.Module]
        The block class used repeatedly in the module. It should not be split by any sharding.
    """
    with open(config_fpath, "r") as f:
        config = json.load(f)
    config["task"] = config["task"].lower().strip()
    tokenizer = Tokenizer(MSA_ALPHABET_PLUS)
    task = TaskType(config["task"].lower().strip())

    # create the model
    model, block = create_model(
        task, config["model_type"], config["model_config"], tokenizer.mask_id.item()
    )

    # add the task-specific wrapper
    aux_loss_weight = config.get("aux_loss_weight", 0.0)
    if task == TaskType.OADM:
        model = OrderAgnosticDiffusionModel(
            model, tokenizer.pad_id, aux_loss_weight=aux_loss_weight
        )
    elif task == TaskType.LM:
        model = ARDiffusionModel(model, aux_loss_weight=aux_loss_weight)
    else:
        raise ValueError(f"Unknown task: {config['task']}")
    return config, tokenizer, model, block


def get_dataloader(
    config: dict, tokenizer: Tokenizer, args: argparse.Namespace
) -> DataLoader:
    if is_amlt():
        data_top_dir = args.data_root or "/ddn/evodiff/"
    else:
        data_top_dir = args.data_root or "/data1/data/"

    dataset = config["dataset"]
    data_dir = os.path.join(data_top_dir, dataset + "/")

    if config["task"] == "oadm":
        collator = OAMaskCollator(
            tokenizer=tokenizer,
            pad_to_multiple_of=config.get("pad_to_multiple_of", None),
        )
    elif config["task"] == "lm":
        collator = LMCollator(
            tokenizer=tokenizer,
            pad_to_multiple_of=config.get("pad_to_multiple_of", None),
            flip_prob=config.get("flip_prob", 0.0),
            fim_prob=config.get("fim_prob", 0.0),
            swap_bos_eos_on_flip=config.get("swap_bos_eos_on_flip", True),
        )
    else:
        raise ValueError(f"Unknown task: {config['task']}")

    # load the dataset
    ds_train = UniRefDataset(data_dir, "train", max_len=config["max_len"])
    train_idx = ds_train.indices

    # create the dataloader
    if args.mini_run:
        tindices = np.arange(
            0, 1000
        )  # np.arange(21546293,31546293,1)#(1000000,21546293, 1)
        train_indices = np.sort(np.random.choice(tindices, 100, replace=False))
        train_sampler = Subset(ds_train, train_indices)
        len_train = train_indices
        dl_train = DataLoader(
            dataset=train_sampler,
            shuffle=True,
            batch_size=1,
            num_workers=1,
            collate_fn=collator,
        )
    else:
        metadata = np.load(os.path.join(data_dir, "lengths_and_offsets.npz"))
        len_train = np.minimum(metadata["ells"][train_idx], config["max_len"])
        if "uniref50" in dataset:
            train_sortish_sampler = SortishSampler(
                len_train, config["bucket_size"], num_replicas=WORLD_SIZE, rank=RANK
            )
        elif "uniref90" in dataset:
            with open(os.path.join(data_dir) + "clustered_splits.json") as f:
                clusters = json.load(f)["train"]
            train_sortish_sampler = ClusteredSortishSampler(
                len_train,
                clusters,
                config["bucket_size"],
                num_replicas=WORLD_SIZE,
                rank=RANK,
            )
        train_sampler = ApproxBatchSampler(
            train_sortish_sampler,
            config["max_tokens"],
            config["max_batch_size"],
            len_train,
            batch_mult=8,
        )

        dl_train = DataLoader(
            dataset=ds_train,
            batch_sampler=train_sampler,
            num_workers=8,
            collate_fn=collator,
            pin_memory=True,
        )

    return dl_train


def seed_everything(seed: int) -> None:
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)




In [3]:
parser = argparse.ArgumentParser()
parser.add_argument("config_fpath")
parser.add_argument(
    "out_fpath",
    type=str,
    nargs="?",
    default=os.getenv("AMLT_OUTPUT_DIR", "/tmp") + "/",
)
parser.add_argument("data_root", type=str, nargs="?", default=None)
parser.add_argument(
    "--mini_run", action="store_true"
)  # Set to True if running on subset of data
parser.add_argument("--checkpoint_freq", type=int, default=2000)  # in steps
parser.add_argument(
    "--random_seed", type=int, default=0
)  # lambda reweighting term from Austin D3PM
parser.add_argument("--dtype", type=str, default="bfloat16")
parser.add_argument("--verbose", action="store_true")
parser.add_argument("--no_wandb", action="store_true")
parser.add_argument("--last_step", default=-1, type=int)

fake_args = [
    "jamba3B-uniref50.json",       # config_fpath
    "/tmp/output/",        # out_fpath
    "../data/",  # data_root
    # "--mini_run",
    "--no_wandb"
]

args = parser.parse_args(fake_args)

In [4]:
print(
    f"Starting job on rank {RANK} with local rank {LOCAL_RANK} and world size {WORLD_SIZE}"
)
seed_everything(args.random_seed)

# dist.init_process_group(backend="nccl")
# get the config, tokenizer, and model
if args.verbose:
    print("Initializing model...", RANK)
config, tokenizer, model, blk_types = load_config_and_model(args.config_fpath)
if RANK == 0:
    if args.no_wandb:
        wandbmode = "disabled"
    else:
        wandbmode = "online"
    wandb.init(config=config, mode=wandbmode)
if args.verbose:
    print("Done initializing model.", RANK)

# store the command line args in the config and dump to disk
config["dtype"] = args.dtype
config["random_seed"] = args.random_seed
config["world_size"] = WORLD_SIZE
if RANK == 0:
    os.makedirs(args.out_fpath, exist_ok=True)
    with open(os.path.join(args.out_fpath, "config.json"), "w") as f:
        json.dump(config, f)

# training dtype and local device
dtype = {
    "float32": torch.float32,
    "float16": torch.float16,
    "bfloat16": torch.bfloat16,
}[args.dtype]

padding_idx = tokenizer.pad_id  # PROTEIN_ALPHABET.index(PAD)
if RANK == 0:
    print("Using {} as padding index".format(padding_idx))
    print("Using {} as masking index".format(tokenizer.mask_id))
    print(
        f"Model has {sum(p.numel() for p in model.parameters())} trainable parameters."
    )
if args.verbose:
    print("Moving and sharding model...", RANK)
# set the default device
torch.cuda.set_device(LOCAL_RANK)


if args.verbose:
    print("Initializing data...", RANK)
dl_train = get_dataloader(config, tokenizer, args)

Starting job on rank 0 with local rank 0 and world size 1


/home/samirchar/miniconda3/envs/dayhoff/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The fast path is not available because on of `(selective_state_update, selective_scan_fn, causal_conv1d_fn, causal_conv1d_update, mamba_inner_fn)` is None. To install follow https://github.com/state-spaces/mamba/#installation and https://github.com/Dao-AILab/causal-conv1d. If you want to use the naive implementation, set `use_mamba_kernels=False` in the model config
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Using 30 as padding index
Using 28 as masking index
Model has 2981431616 trainable parameters.


In [ ]:
# Documentation create dataset loading script: https://huggingface.co/docs/datasets/en/dataset_script
# Basic dataset creation and benefits of using HF datasets (first paragraph): https://huggingface.co/docs/datasets/en/create_dataset
# Another good (Japanese) tutorial for create_dataset: https://zenn.dev/mjun0812/articles/cfe4b7346ba6b4

from huggingface_hub import login, ModelCard
from dotenv import load_dotenv
load_dotenv()
login(token=os.getenv("HF_TOKEN"))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [23]:
dl_train_iter = iter(dl_train)

In [24]:
dl_train_iter_obs = next(dl_train_iter)

In [30]:
dl_train_iter_obs[0].shape,dl_train_iter_obs[1].shape

(torch.Size([1000, 80]), torch.Size([1000, 80]))

In [ ]:
dl_train_iter_obs[0].shape

In [18]:
for batch in train_loader:
    print(batch)
    break

{'label': tensor([4, 1, 3, 3]), 'text': ["dr. goldberg offers everything i look for in a general practitioner.  he's nice and easy to talk to without being patronizing; he's always on time in seeing his patients; he's affiliated with a top-notch hospital (nyu) which my parents have explained to me is very important in case something happens and you need surgery; and you can get referrals to see specialists without having to see him first.  really, what more do you need?  i'm sitting here trying to think of any complaints i have about him, but i'm really drawing a blank.", "Unfortunately, the frustration of being Dr. Goldberg's patient is a repeat of the experience I've had with so many other doctors in NYC -- good doctor, terrible staff.  It seems that his staff simply never answers the phone.  It usually takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  Yo

In [ ]:
import datasets
import os.path as osp
from torch.utils.data import Dataset
from dayhoff.datasets import UniRefDataset
from dataclasses import dataclass
from typing import Literal

_DESCRIPTION = """\
Dayhoff dataset
"""

# TODO: Add a link to an official homepage for the dataset here
_HOMEPAGE = "https://github.com/microsoft/dayhoff/tree/main"

# TODO: Add the licence for the dataset here if you can find it
_LICENSE = ""

#TODO: Add citation
_CITATION = ""

# TODO: Add link to the official dataset URLs here
# The HuggingFace Datasets library doesn't host the datasets but only points to the original files.
# This can be an arbitrary nested dict/list of URLs (see below in `_split_generators` method)
_URLS = {
    
}

@dataclass
class SequencesConfig(datasets.BuilderConfig):
        '''Congif for sequence generation'''
        dataset: Literal["uniref50_202401", "uniref90_202401", "gigaref"]
        name: str = "sequence"
        max_seq_len: int = 2048

@dataclass
class MSAConfig(datasets.BuilderConfig):
        '''Congif for MSA generation'''
        dataset: Literal["uniref50_202401", "gigaref_with_singletons", "gigaref_no_singletons"] #TODO: complete all possible datasets
        name: str = "msa"
        #TODO: Complete all arguments by looking at dataset class, possibly OpenProteinDataset


# Name of the dataset usually matches the script name with CamelCase instead of snake_case
class Dayhoff(datasets.GeneratorBasedBuilder):
    """TODO: Short description of my dataset."""
    
    VERSION = datasets.Version("1.1.0")
    DEFAULT_CONFIG_NAME = "sequence"  # It's not mandatory to have a default configuration. Just use one if it make sense.

    # This is an example of a dataset with multiple configurations.
    # If you don't want/need to define several sub-sets in your dataset,
    # just remove the BUILDER_CONFIG_CLASS and the BUILDER_CONFIGS attributes.

    # If you need to make complex sub-parts in the datasets with configurable options
    # You can create your own builder configuration class to store attribute, inheriting from datasets.BuilderConfig
    # BUILDER_CONFIG_CLASS = MyBuilderConfig

    # You will be able to load one or the other configurations in the following list with
    # data = datasets.load_dataset('my_dataset', 'first_domain')
    # data = datasets.load_dataset('my_dataset', 'second_domain')
    BUILDER_CONFIGS = [
        SequencesConfig(version=VERSION, description="sequence generation"),
        MSAConfig(version=VERSION, description="MSA generation"),

    ]

    

    def _info(self):
        # TODO: This method specifies the datasets.DatasetInfo object which contains informations and typings for the dataset
        # Maybe add sequence and MSA configs?
        if self.config.name == "sequence":
            features = datasets.Features(
                {
                    "sequence": datasets.Value("string")
                }
            )

            homepage= "" # TODO: add HF homepage

        if self.config.name == "msa":
             raise NotImplementedError("MSA config not implemented yet") #TODO: Implement MSA config

        return datasets.DatasetInfo(
            # This is the description that will appear on the datasets page.
            # description=_DESCRIPTION,
            # This defines the different columns of the dataset and their types
            features=features,  
            # License for the dataset if available
            license=_LICENSE,
            # Citation for the dataset
            citation=_CITATION,
            
            supervised_keys=None
        )

    def _split_generators(self, dl_manager):
        # TODO: This method is tasked with downloading/extracting the data and defining the splits depending on the configuration
        # If several configurations are possible (listed in BUILDER_CONFIGS), the configuration selected by the user is in self.config.name

        # dl_manager is a datasets.download.DownloadManager that can be used to download and extract URLS
        # It can accept any type or nested list/dict and will give back the same structure with the url replaced with path to local files.
        # By default the archives will be extracted and a path to a cached folder where they are extracted is returned instead of the archive
        if self.config.name == "sequence":
            
            file_paths = dl_manager.download({
                 'consensus':'https://huggingface.co/datasets/samirchar/dayhoff/consensus_sample.fasta',
                 'splits':'https://huggingface.co/datasets/samirchar/dayhoff/splits.json',
                 'splits':'https://huggingface.co/datasets/samirchar/dayhoff/splits_and_offsets.npz'
            })

            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={
                        "data_dir": "", #TODO: Add data dir
                        "split": "train",
                        "max_len":self.config.max_seq_len, #TODO: can max_len and split be passed as arguments?
                        "split_file":None
                    },
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={
                        "data_dir": "", #TODO: Add data dir
                        "split": "valid",
                        "max_len":self.config.max_seq_len, #TODO: can max_len and split be passed as arguments?
                        "split_file":None
                    },
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.TEST,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={
                        "data_dir": "", #TODO: Add data dir
                        "split": "test",
                        "max_len":self.config.max_seq_len, #TODO: can max_len and split be passed as arguments?
                        "split_file":None
                    },
                ),
            ]

    # method parameters are unpacked from `gen_kwargs` as given in `_split_generators`
    def _generate_examples(self, data_dir, split, max_len, split_file):
        # TODO: This method handles input defined in _split_generators to yield (key, example) tuples from the dataset.
        # The `key` is for legacy reasons (tfds) and is not important in itself, but must be unique for each example.

        if self.config.name == "sequence":
            dataset = UniRefDataset(data_dir = data_dir, split = split, max_len = max_len, split_file = split_file)
            for key, row in enumerate(dataset):
                yield key, {
                    "sequence": row
                }

In [11]:
with open('../data/uniref50_202401/splits.json','r') as f:
    splits = json.load(f)

offsets = np.load('../data/uniref50_202401/lengths_and_offsets.npz')

In [15]:
offsets['name_offsets']

array([          0,       45467,       85257, ..., 25814319433,
       25814319555, 25814319685])

In [19]:
with open('../data/uniref50_202401/splits.json','r') as f:
    splits = json.load(f)

In [20]:
splits.keys()

dict_keys(['train', 'test', 'valid', 'rtest'])